In [5]:
# !pip3 install requests
# !pip3 install dotenv

In [15]:
from dotenv import load_dotenv

import os

load_dotenv()
PROJECT_TOKEN = os.environ.get("PROJECT_TOKEN")
PROJECT_KEY = os.environ.get("PROJECT_KEY") 

In [11]:
import requests

# Define the endpoint URL
url = "https://api-main-poc.aiml.asu.edu/token"

# Set your project key as the Authorization Bearer Token
headers = {
    "Authorization": PROJECT_TOKEN,
    "Content-Type": "application/json"
}

# Define the request payload
payload = {
    "method": "generate_token",
    "details": {
        "asurite": "sewing12"
    }
}

# Send the POST request
response = requests.post(url, headers=headers, json=payload)

# Check the response
if response.status_code == 200:
    print("Token generated successfully!")
    # print("Response:", response.json())
    
else:
    print("Failed to generate token.")
    print("Status Code:", response.status_code)
    print("Response:", response.text)

Token generated successfully!


In [12]:
import json

json_string = response.json()
made_USER_TOKEN = json_string['token']

In [13]:
import pandas as pd
half_g = pd.read_csv('half_golden.csv')
half_g = half_g[['section', 'title', 'file_name', 'document_type', 'page', 'total_pages', 'context', 'question', 'answer']]
half_g.rename(columns={'answer':'golden_answer'}, inplace = True)
half_g['model'] = 'Claude35'
half_g['Retrieval_Type'] = 'Neighbor' ## Neighbor, Chunk, Document
half_g['Top_K'] = 3
half_g['Response'] = ""
half_g.head()

,section,title,file_name,document_type,page,total_pages,context,question,golden_answer,model,Retrieval_Type,Top_K,Response
0,Results,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,8,12,['Fig. 4. A Euclidian single cluster dendrogra...,What are the limitations of the correlation m...,The text states that while a correlation matr...,Claude35,Neighbor,3,
1,Results,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,8,12,['Fig. 4. A Euclidian single cluster dendrogra...,"Based on the hierarchical cluster analysis, h...",The text states that the number of clusters w...,Claude35,Neighbor,3,
2,Results,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,7,12,['The word stems have meaning based on their a...,What specific methods were used to analyze th...,The paper mentions several methods used to an...,Claude35,Neighbor,3,
3,Results,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,6,12,['Fig. 2. Bar chart of the top 25 occurring wo...,How does the bi-plot in Fig. 3 provide insigh...,The bi-plot visually represents the relations...,Claude35,Neighbor,3,
4,Abstract,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,1,12,['Text Mining for Procedure Level Primitives i...,How does this text mining approach relate to ...,The text mining approach is used to identify ...,Claude35,Neighbor,3,


In [14]:
import requests

# Define the endpoint URL
url = "https://api-main-poc.aiml.asu.edu/queryV2"

# Set your Authorization Bearer Token (use the project token if you have it)
headers = {
    "Authorization": made_USER_TOKEN,
    "Content-Type": "application/json"
}

# Define the request payload
payload = {
    "project_id": PROJECT_KEY,
    "query": half_g['question'].iloc[0],
    "enable_search": True,
    "enable_history": False,
    "history": []
}

# Send the POST request
response = requests.post(url, headers=headers, json=payload)

# Check the response
if response.status_code == 200:
    print("Request successful!")
    print("Response:", response.json())
else:
    print("Failed to make request.")
    print("Status Code:", response.status_code)
    print("Response:", response.text)

Request successful!
Response: {'response': 'Based on the information provided, there are a few key limitations of using a correlation matrix to visualize relationships between procedures, and ways the correlation network addresses these:\n\n1. Visualization difficulty: The passage notes that while a correlation matrix contains a lot of information, "visualization of that data can be difficult and chaotic." A correlation matrix with many variables can be overwhelming to interpret visually.\n\n2. Limited insight into structure: The matrix alone may not clearly reveal underlying groupings or structures in the data.\n\n3. Difficulty seeing strength of relationships: The magnitude of correlations is not easily discernible in a large matrix.\n\nThe correlation network addresses these limitations in the following ways:\n\n1. Improved visualization: The network provides a clearer visual representation of the relationships between procedures compared to a matrix.\n\n2. Reveals groupings: The ne

In [22]:
## build code to run the half golden questions against the project space where you alter top K, then measure against the hlf golden answers
question_response = response.json()
print(question_response['response'])


Based on the information provided, there are a few key points about the limitations of the correlation matrix and how the correlation network addresses them:

1. Visualization difficulty: The knowledge base states that "While a matrix contains a lot of information, visualization of that data can be difficult and chaotic." This indicates that a correlation matrix, while containing useful information, can be challenging to interpret visually.

2. Network visualization: To address this limitation, "a network was constructed to better visualize the correlation relationships between the stem words". The correlation network provides a more intuitive visual representation of the relationships.

3. Strength of correlations: In the network, "The thickness of the lines between the stem word nodes denotes the strength of the correlation." This allows for easy identification of strong vs weak correlations.

4. Direction of correlations: "In addition to line thickness, the colors of the lines indic

In [23]:
print(half_g['question'].iloc[0])

 What are the limitations of the correlation matrix in visualizing the relationships between procedures, and how does the correlation network address these limitations?


In [24]:
print(half_g['golden_answer'].iloc[0])

 The text states that while a correlation matrix contains a lot of information, visualizing it can be difficult and chaotic. This is likely because the matrix represents a complex network of relationships, making it difficult to identify patterns or trends. The correlation network addresses these limitations by providing a more intuitive visual representation. The thickness of the lines represents the strength of the correlation, and the colors indicate whether the correlation is positive or negative, making it easier to understand the relationships between procedures. This allows for a more effective visualization of the data and helps to identify potential groupings of procedures.


In [25]:
half_g.loc[0, 'Response'] = question_response['response']

In [26]:
half_g.head()

,section,title,file_name,document_type,page,total_pages,context,question,golden_answer,model,Retrieval_Type,Top_K,Response
0,Results,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,8,12,['Fig. 4. A Euclidian single cluster dendrogra...,What are the limitations of the correlation m...,The text states that while a correlation matr...,Claude35,Neighbor,3,"Based on the information provided, there are a..."
1,Results,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,8,12,['Fig. 4. A Euclidian single cluster dendrogra...,"Based on the hierarchical cluster analysis, h...",The text states that the number of clusters w...,Claude35,Neighbor,3,
2,Results,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,7,12,['The word stems have meaning based on their a...,What specific methods were used to analyze th...,The paper mentions several methods used to an...,Claude35,Neighbor,3,
3,Results,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,6,12,['Fig. 2. Bar chart of the top 25 occurring wo...,How does the bi-plot in Fig. 3 provide insigh...,The bi-plot visually represents the relations...,Claude35,Neighbor,3,
4,Abstract,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,1,12,['Text Mining for Procedure Level Primitives i...,How does this text mining approach relate to ...,The text mining approach is used to identify ...,Claude35,Neighbor,3,
